<a href="https://colab.research.google.com/github/haminhtien99/re3-pytorch/blob/master/home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch
# !git clone https://github.com/danielgordon10/re3-pytorch

/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch


###  Chuẩn bị dataset dùng để huấn luyện mạng


Gồm 2 thư mục images và labels. Mỗi thư mục chia thành hai thư mục con là train/test
* images --- train/test ---files .jpg
* labels --- train/test
  * files .txt: tên các file trong images
  * files .npy: Chứa thông tin về boungding box và các thuộc tính class, video, imgNum

In [7]:
import cv2
import numpy as np
import glob
import xml.etree.ElementTree as ET
import time
import random
import os
import sys
import shutil

In [38]:
source_folder = "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/demo/data/"
dest_folder = "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/data/"
jpg_files = [f for f in os.listdir(source_folder) if f.endswith(".jpg")]
jpg_files.sort()
num_train = len(jpg_files)//3 * 2

img_train = jpg_files[:num_train]
img_test = jpg_files[num_train:]
for img in img_train:
    source_path = source_folder + img
    dest_path = dest_folder + "/images/train/"
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    shutil.copy(source_path, dest_path)
for img in img_test:
    source_path = source_folder + img
    dest_path = dest_folder + "/images/test/"
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    shutil.copy(source_path, dest_path)



In [39]:
print(img_test)

['00000455.jpg', '00000456.jpg', '00000457.jpg', '00000458.jpg', '00000459.jpg', '00000460.jpg', '00000461.jpg', '00000462.jpg', '00000463.jpg', '00000464.jpg', '00000465.jpg', '00000466.jpg', '00000467.jpg', '00000468.jpg', '00000469.jpg', '00000470.jpg', '00000471.jpg', '00000472.jpg', '00000473.jpg', '00000474.jpg', '00000475.jpg', '00000476.jpg', '00000477.jpg', '00000478.jpg', '00000479.jpg', '00000480.jpg', '00000481.jpg', '00000482.jpg', '00000483.jpg', '00000484.jpg', '00000485.jpg', '00000486.jpg', '00000487.jpg', '00000488.jpg', '00000489.jpg', '00000490.jpg', '00000491.jpg', '00000492.jpg', '00000493.jpg', '00000494.jpg', '00000495.jpg', '00000496.jpg', '00000497.jpg', '00000498.jpg', '00000499.jpg', '00000500.jpg', '00000501.jpg', '00000502.jpg', '00000503.jpg', '00000504.jpg', '00000505.jpg', '00000506.jpg', '00000507.jpg', '00000508.jpg', '00000509.jpg', '00000510.jpg', '00000511.jpg', '00000512.jpg', '00000513.jpg', '00000514.jpg', '00000515.jpg', '00000516.jpg', '000005

In [ ]:
train_labels_path = dest_folder + "labels/train/"
test_labels_path = dest_folder  + "labels/test/"
if not os.path.exists(train_labels_path):
    os.makedirs(train_labels_path)
if not os.path.exists(test_labels_path):
    os.makedirs(test_labels_path)
with open(train_labels_path+ "images.txt", 'w') as file:
    for img in img_train:
        file.write(img + '\n')
with open(test_labels_path+ "images.txt", 'w') as file:
    for img in img_test:
        file.write(img + '\n')

In [ ]:
labels_txt = source_folder +"labels.txt"
with open(labels_txt, 'r') as file:
    lines = file.read().splitlines()
bboxes = []
imNum = 1
for line in lines[:num_train]:
    # print(line)
    line = [int(float(number)) for number in line.split()]

    videoId = 0
    trackId = 0
    classId = 0
    occl = 0
    bbox = [line[1], line[2], line[3], line[4], videoId, trackId, imNum, classId, occl]
    bboxes.append(bbox)
    imNum += 1
bboxes = np.array(bboxes)
# print(bboxes)

if not os.path.exists(train_labels_path):
    os.makedirs(train_labels_path)
np.save(train_labels_path + "/labels.npy", bboxes)


In [ ]:
bboxes = []
imNum = 1
for line in lines[num_train:]:
    # print(line)
    line = [int(float(number)) for number in line.split()]
    # imNum =  line[0]
    videoId = 0
    trackId = 0
    classId = 0
    occl = 0
    bbox = [line[1], line[2], line[3], line[4], videoId, trackId, imNum, classId, occl]
    bboxes.append(bbox)
    imNum+= 1
bboxes = np.array(bboxes)

if not os.path.exists(test_labels_path):
    os.makedirs(test_labels_path)
np.save(test_labels_path + "/labels.npy", bboxes)

### Training


In [4]:
!python3 training/get_datasets.py

In [11]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training
!python unrolled_solver.py -rtc -n 2 -b 64

/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training
2023-10-10 13:34:14.262702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 13:34:15.851551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
non-resource variables are not supported in the long term
#my_dataset keys: 284
Restoring
restoring from /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/logs/checkpoints
restoring from /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/logs/checkpoints/iteration_0001222.pt
Got exception while trying to restore
Traceback (most recent call last):
  File "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/re3_utils/pytorch_util/pytorch_util_functions.py", l

### Tracking


In [16]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch
import os.path
import sys
import time

import cv2
import numpy as np


from re3_utils.util import bb_util
from re3_utils.util import im_util
from re3_utils.pytorch_util import pytorch_util_functions as pt_util

from tracker.network import Re3Net, Re3SmallNet

# Network Constants
from constants import CROP_SIZE
from constants import CROP_PAD
from constants import LOG_DIR
from constants import MAX_TRACK_LENGTH
from constants import USE_SMALL_NET

SPEED_OUTPUT = True


class Re3Tracker(object):
    def __init__(self, gpu_id=0):
        gpu_id = pt_util.setup_devices(gpu_id)[0]

        if USE_SMALL_NET:
            self.network = Re3SmallNet(gpu_id)
            pt_util.restore_from_folder(self.network, os.path.join(LOG_DIR, "checkpoints_small"))
        else:
            self.network = Re3Net(gpu_id)
            pt_util.restore_from_folder(self.network, os.path.join(LOG_DIR, "checkpoints"))
        self.network.to(gpu_id)
        self.network.eval()


        self.tracked_data = {}

        self.t_time = 0
        self.total_forward_count = -1

    # unique_id{str}: A unique id for the object being tracked.
    # image{str or numpy array}: The current image or the path to the current image.
    # starting_box{None or 4x1 numpy array or list}: 4x1 bounding box in X1, Y1, X2, Y2 format.
    def track(self, unique_id, image, starting_box=None):
        start_time = time.time()

        if type(image) == str:
            image = cv2.imread(image)[:, :, ::-1]
        else:
            image = image.copy()

        image_read_time = time.time() - start_time

        if starting_box is not None:
            lstm_state = None
            past_bbox = np.array(starting_box)  # turns list into numpy array if not and copies for safety.
            prev_image = image
            original_features = None
            forward_count = 0
        elif unique_id in self.tracked_data:
            lstm_state, past_bbox, prev_image, original_features, forward_count = self.tracked_data[unique_id]
        else:
            raise Exception("Unique_id %s with no initial bounding box" % unique_id)

        cropped_input0, past_b_box_padded = im_util.get_cropped_input(prev_image, past_bbox, CROP_PAD, CROP_SIZE)
        cropped_input1, _ = im_util.get_cropped_input(image, past_bbox, CROP_PAD, CROP_SIZE)
        # import pdb
        # pdb.set_trace()

        image_input = pt_util.from_numpy((np.stack([cropped_input0, cropped_input1])))
        raw_output = self.network(image_input, lstm_state)
        raw_output = pt_util.to_numpy_array(raw_output)
        lstm_state = self.network.lstm_state
        if forward_count == 0:
            original_features = [var.clone().detach() for var in self.network.lstm_state]

        prev_image = image

        # Shift output box to full image coordinate system.
        output_box = bb_util.from_crop_coordinate_system(raw_output.squeeze() / 10.0, past_b_box_padded, 1, 1)
        # import pdb
        # pdb.set_trace()
        if forward_count > 0 and forward_count % MAX_TRACK_LENGTH == 0:
            cropped_input, _ = im_util.get_cropped_input(image, output_box, CROP_PAD, CROP_SIZE)
            image_input = pt_util.from_numpy(np.tile(cropped_input[np.newaxis, ...], (2, 1, 1, 1)))
            self.network(image_input, original_features)
            lstm_state = self.network.lstm_state

        forward_count += 1
        self.total_forward_count += 1

        if starting_box is not None:
            # Use label if it's given
            output_box = np.array(starting_box)

        self.tracked_data[unique_id] = (lstm_state, output_box, image, original_features, forward_count)
        end_time = time.time()
        if self.total_forward_count > 0:
            self.t_time += end_time - start_time - image_read_time
        if SPEED_OUTPUT and self.total_forward_count % 100 == 0:
            print("Current tracking speed:   %.3f FPS" % (1 / (end_time - start_time - image_read_time)))
            print("Current image read speed: %.3f FPS" % (1 / (image_read_time)))
            print("Mean tracking speed:      %.3f FPS\n" % (self.total_forward_count / max(0.00001, self.t_time)))
        return output_box


/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch


In [ ]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch
import glob
import os.path
import sys

import cv2

# basedir = os.path.dirname(__file__)
# sys.path.append(os.path.abspath(os.path.join(basedir, os.path.pardir)))
from tracker import re3_tracker


tracker = Re3Tracker()




In [44]:
test_folder = "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/data/images/test"
test_files = []
test_files = [test_folder +"/"+ f for f in os.listdir(test_folder) if f.endswith(".jpg")]
test_files.sort()
initial_bbox = [154, 112, 254, 238]
image = cv2.imread(test_files[0])

tracker.track("ball", test_files[0], initial_bbox)
for image_path in test_files:
    image = cv2.imread(image_path)
    # Tracker expects RGB, but opencv loads BGR.
    imageRGB = image[:, :, ::-1]
    bbox = tracker.track("ball", imageRGB)
    print(bbox)
    # cv2.rectangle(image, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), [0, 0, 255], 2)
    # cv2.imshow("Image", image)
    # cv2.waitKey(1)

RuntimeError: ignored